In [1]:
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import os
import sys
import time
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np

In [2]:
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [3]:
def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")

In [4]:
cfg = pedestrian
batch_size = 1
gamma = 0.1
dataset_root = VOC_ROOT
save_folder = 'weights/'
basenet = 'vgg16_reducedfc.pth'
lr = 1e-3
momentum = 0.9
weight_decay = 5e-4

In [5]:
def train():
    
    #Dataset
    dataset = VOCDetection(root = dataset_root,transform=SSDAugmentation(cfg['min_dim'], MEANS))
    
    #Building ssd
    ssd_net = build_ssd('train', cfg['min_dim'], cfg['num_classes'])
    net = ssd_net
    
    #Activating Cuda
    net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True
    
    #Loading weights
    vgg_weights = torch.load(save_folder + basenet)
    print('Loading base network...')
    ssd_net.vgg.load_state_dict(vgg_weights)
    print('Initializing weights...')
    # initialize newly added layers' weights with xavier method
    ssd_net.extras.apply(weights_init)
    ssd_net.loc.apply(weights_init)
    ssd_net.conf.apply(weights_init)
    
    net = net.cuda()
    
    #Loss and optimization
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum = momentum,weight_decay = weight_decay)
    criterion = MultiBoxLoss(cfg['num_classes'], 0.5, True, 0, True, 3, 0.5,False, True)
    
    net.train()
    # loss counters
    loc_loss = 0
    conf_loss = 0
    epoch = 0
    print('Loading the dataset...')

    epoch_size = len(dataset) // batch_size
    print('Training SSD on:', dataset.name)
    
    step_index = 0
    
    #Loading Dataset    
    data_loader = data.DataLoader(dataset, batch_size, num_workers=4, shuffle=True,
                                  collate_fn=detection_collate, pin_memory=True)
    
    # create batch iterator
    batch_iterator = iter(data_loader)
    for iteration in range(0, cfg['max_iter']):
        if iteration in cfg['lr_steps']:
            step_index += 1
            adjust_learning_rate(optimizer, gamma, step_index)

        # load train data
        try:
            images, targets = next(batch_iterator)
        except StopIteration:
            batch_iterator = iter(data_loader)
            images, targets = next(batch_iterator)
        
        images = Variable(images.cuda())
        with torch.no_grad():
            targets = [Variable(ann.cuda()) for ann in targets]
        
        # forward
        t0 = time.time()
        out = net(images)
        
        # backprop
        optimizer.zero_grad()
        loss_l, loss_c = criterion(out, targets)
        loss = loss_l + loss_c
        loss.backward()
        optimizer.step()
        t1 = time.time()
        loc_loss += loss_l.item()
        conf_loss += loss_c.item()

        if iteration % 10 == 0:
            print('timer: %.4f sec.' % (t1 - t0))
            print('iter ' + repr(iteration) + ' || Loss: %.4f ||' % (loss.item()), end=' ')

        
        if iteration != 0 and iteration % 5000 == 0:
            print('Saving state, iter:', iteration)
            torch.save(ssd_net.state_dict(), 'weights/ssd300_VOC_' + repr(iteration) + '.pth')
    
    torch.save(ssd_net.state_dict(),
               args.save_folder + '' + args.dataset + '.pth')

In [6]:
def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = lr * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [7]:
def xavier(param):
    init.xavier_uniform_(param)

In [8]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()

In [9]:
if __name__ == '__main__':
    train()

Loading base network...
Initializing weights...
Loading the dataset...
Training SSD on: VOC0712


/home/krxat/anaconda3/envs/torch/lib/python3.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


timer: 16.5736 sec.
iter 0 || Loss: 19.8159 || 

KeyboardInterrupt: 